In [1]:
from langchain.docstore.document import Document
import os
import requests
import json
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import ChatOllama
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, RecursiveJsonSplitter

In [2]:
# preprocess data
import re
with open("civil_code_fixed.txt", 'r', encoding='utf-8-sig') as file:
    text = file.read()

data = []
newText = text.split(".\nArticle ")
data.append(Document(page_content=newText[0], metadata={"source": "local"}))
for t in newText[1:]:
    print(t)
    doc = Document(page_content="Article "+ t, metadata={"source": "local"})  
    print("______________________")
    data.append(doc)

# 
# Rule468

2  
(1) This Code must be construed so as to honor the dignity of individuals and the essential equality of both sexes
______________________
3  
(1) The enjoyment of private rights commences at birth.
(2) Unless otherwise prohibited by applicable laws, regulations, or treaties, foreign nationals enjoy private rights
______________________
3-2  
(1) If the person making a juridical act did not have mental capacity when manifesting the relevant intention, the juridical act is void
______________________
4  
(1) The age of majority is 20 years of age
______________________
5  
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act; provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
(3) Notwithstanding the provisions of paragraph (1), a minor may freely dispose of property t

In [3]:
file.close()

In [19]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 250, chunk_overlap = 0
)
with open("civil_code.txt", 'r', encoding="utf8") as file:
    text = file.read()

# print(text, len(text), type(text))

# with open('testModified.json', 'r', encoding="utf8") as file:
#     dev_dataset = json.load(file)

# json_splitter = RecursiveJsonSplitter(max_chunk_size=300)
# json_chucks = json_splitter.split_json(json_data=dev_dataset)

# texts = json_splitter.split_text(json_data=dev_dataset)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap=200
)

doc_splits = text_splitter.create_documents([text])


# data = json.dumps(dev_dataset)
# print(data)
# print(text_splitter.split_documents([data]))


In [50]:
print(doc_splits)
# for i in doc_splits:
#     print("STUFF:", i)


[Document(metadata={}, page_content='ï»¿Civil Code (Part I, Part II and Part III)\nPart I General Provisions\nChapter I Common Provisions\n(Fundamental Principles)\nArticle 1  (1) Private rights must be congruent with the public welfare.\n(2) The exercise of rights and performance of duties must be done in good faith.\n(3) Abuse of rights is not permitted.\n(Standards for Construction)\nArticle 2  This Code must be construed so as to honor the dignity of individuals and the essential equality of both sexes.\nChapter II Persons\nSection 1 Capacity to Hold Rights\nArticle 3  (1) The enjoyment of private rights commences at birth.\n(2) Unless otherwise prohibited by applicable laws, regulations, or treaties, foreign nationals enjoy private rights.\nSection 2 Mental Capacity\nArticle 3-2  If the person making a juridical act did not have mental capacity when manifesting the relevant intention, the juridical act is void.\nSection 3 Capacity to Act\n(Age of Majority)\nArticle 4  The age of m

In [12]:
local_llm = "llama3.2"
llm = ChatOllama(model = local_llm, temperature = 0)
vectorstore = SKLearnVectorStore.from_documents(
    documents=data,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")
)

retriever = vectorstore.as_retriever(k=3)

In [13]:
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [22]:
from langchain_core.messages import HumanMessage, SystemMessage

router_instructions = """You are an expert at routing a user question to a vectorstore or web search.

The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.

Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.

Return JSON with single key, datasource, that is 'websearch' or 'vectorstore' depending on the question."""
# query = "A special provision that releases warranty can be made, but in that situation, when there are rights that the seller establishes on his/her own for a third party, the seller is not released of warranty."
# query = "There is a limitation period on pursuance of warranty if there is restriction due to superficies on the subject matter, but there is no restriction on pursuance of warranty if the seller's rights were revoked due to execution of the mortgage."
# query = "A compulsory auction is also a sale, so warranty is imposed the same as for an ordinary sale."
# query = "In cases where an individual rescues another person from getting hit by a car by pushing that person out of the way, causing the person's luxury kimono to get dirty, the rescuer does not have to compensate damages for the kimono."
query = "Usufructuary rights are only established for real estate."

question = [HumanMessage(content="Is this true or false? " + query)]
# test_vector_store = llm_json_mode.invoke([SystemMessage(content=router_instructions)]+ question)
# test_vector_store.content

In [14]:
### Retrieval Grader

# Doc grader instructions
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

Think carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

# Test
# question = "A special provision that releases warranty can be made, but in that situation, when there are rights that the seller establishes on his/her own for a third party, the seller is not released of warranty."
question_needed = "What are the facts of this case? " + query
docs = retriever.invoke(question_needed)
doc_txt = docs[1].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
# result = llm_json_mode.invoke(
#     [SystemMessage(content=doc_grader_instructions)]
#     + [HumanMessage(content=doc_grader_prompt_formatted)]
# )
# json.loads(result.content)

In [87]:
# test_instructions = """You are a lawer. Here are the only rule book you get to use: \n\n {context} \n\nYou may only use what is in the rulebook to answer the question. what is the type of this legal statement?: \n\n {statement}. Return only what is asked."""
prompt = """You are a lawer. Here are the only civil code book you get to use: \n\n {context} \n\nYou may only use what is in the given book to answer the question. The statement is: {statement}."""
test_instructions = prompt + """If part I of the book is 'General Provisions', what are all the parts of the code book. Return only what is asked."""


In [88]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

docs = format_docs(data)
rag_prompt_formatted = test_instructions.format(context=text, statement = query)
# generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])


In [89]:
generation = llm.invoke(rag_prompt_formatted)
print(generation.content)

I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII


In [59]:
generation = llm.invoke(rag_prompt_formatted)
print(generation.content)

Here are the relevant articles:

* Article 717-1: "Usufructuary rights may be established for movable property."
* No other article directly supports or contradicts the statement.

However, since there is no direct support, and usufructuary rights can indeed be established for real estate (as implied by the statement), I would say that:

Article 717-1 does not contradict the statement.


In [113]:
text_1 = "Break this query: \n\n {query} \n\n down to multiple easier-to-process queries for a RAG model to retrieve answer, based on these articles: \n\n {context} \n\n "
text_1 = "Given these articles: \n\n {context} \n\nAnd this query: \n\n{query}\n\nWhich articles can be use to verify this query?"
# text_1 += question_needed
text_2 = text_1.format(context = docs, query = query)
print(text_2)

Given these articles: 

 Article 1
(1) Private rights must be congruent with the public welfare.
(2) The exercise of rights and performance of duties must be done in good faith.
(3) Abuse of rights is not permitted

Article 2  
(1) This Code must be construed so as to honor the dignity of individuals and the essential equality of both sexes

Article 3  
(1) The enjoyment of private rights commences at birth.
(2) Unless otherwise prohibited by applicable laws, regulations, or treaties, foreign nationals enjoy private rights

Article 3-2  
(1) If the person making a juridical act did not have mental capacity when manifesting the relevant intention, the juridical act is void

Article 4  
(1) The age of majority is 20 years of age

Article 5  
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act; provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act 

In [114]:
generation = llm.invoke(text_2)
print(generation.content)

To verify whether usufructuary rights are only established for real estate, we need to look at the relevant articles in the provided text. Here are a few articles that might be relevant:

* Article 718: This article states that a possessor of an animal is liable to compensate for damage that the animal inflicts on another person. However, this article does not specifically address usufructuary rights.
* Article 717: This article discusses defects in structures on land and liability for damages caused by such defects. Again, this article does not directly address usufructuary rights.

However, there is no explicit mention of usufructuary rights in the provided text. Usufructuary rights are indeed typically associated with real estate, but they are not explicitly mentioned in the articles listed above.

If you're looking for a more comprehensive answer, I would recommend consulting a specific law or jurisdiction's regulations regarding usufructuary rights, as they may vary depending on t

In [112]:
rag_prompt_formatted = test_instructions.format(context=docs, question=generation.content)
# generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
generation = llm.invoke(rag_prompt_formatted)
print(generation.content)

Here are the article numbers corresponding to the answers:

1. Usufructuary rights: Article 725
2. Who can have usufructuary rights? 
   - A person who has a right in a thing (Article 725)
   - The possessor of an animal (Article 718)


In [33]:
### Generate

# Prompt
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question_needed)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

Based on the provided context, it is false that the rescuer does not have to compensate damages for the kimono. According to Article 191, a possessor in good faith is liable to compensate for loss or damage to the extent they are currently enriched as a result of that loss or damage. However, there is no direct application of this principle to a situation where someone rescues another person from harm and causes them to lose their property.


Break this sentence down to easier-to-process sentence for a RAG model to retrieve answer: Is the next sentence true or false? In cases where an individual rescues another person from getting hit by a car by pushing that person out of the way, causing the person's luxury kimono to get dirty, the rescuer does not have to compensate damages for the kimono.


In [19]:
### Hallucination Grader

# Hallucination grader instructions
hallucination_grader_instructions = """

You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 

(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Score:

A score of yes means that the student's answer meets all of the criteria. This is the highest (best) score. 

A score of no means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader prompt
hallucination_grader_prompt = """FACTS: \n\n {documents} \n\n STUDENT ANSWER: {generation}. 

Return JSON with two two keys, binary_score is 'yes' or 'no' score to indicate whether the STUDENT ANSWER is grounded in the FACTS. And a key, explanation, that contains an explanation of the score."""

# Test using documents and generation from above
hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(
    documents=docs_txt, generation=generation.content
)
result = llm_json_mode.invoke(
    [SystemMessage(content=hallucination_grader_instructions)]
    + [HumanMessage(content=hallucination_grader_prompt_formatted)]
)
json.loads(result.content)

{'binary_score': 'no', 'explanation': ''}

In [18]:
### Answer Grader

# Answer grader instructions
answer_grader_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) The STUDENT ANSWER helps to answer the QUESTION

Score:

A score of yes means that the student's answer meets all of the criteria. This is the highest (best) score. 

The student can receive a score of yes if the answer contains extra information that is not explicitly asked for in the question.

A score of no means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader prompt
answer_grader_prompt = """QUESTION: \n\n {question} \n\n STUDENT ANSWER: {generation}. 

Return JSON with two two keys, binary_score is 'yes' or 'no' score to indicate whether the STUDENT ANSWER meets the criteria. And a key, explanation, that contains an explanation of the score."""

# Test
question = "What are the vision models released today as part of Llama 3.2?"
answer = "The Llama 3.2 models released today include two vision models: Llama 3.2 11B Vision Instruct and Llama 3.2 90B Vision Instruct, which are available on Azure AI Model Catalog via managed compute. These models are part of Meta's first foray into multimodal AI and rival closed models like Anthropic's Claude 3 Haiku and OpenAI's GPT-4o mini in visual reasoning. They replace the older text-only Llama 3.1 models."

# Test using question and generation from above
answer_grader_prompt_formatted = answer_grader_prompt.format(
    question=question, generation=answer
)
result = llm_json_mode.invoke(
    [SystemMessage(content=answer_grader_instructions)]
    + [HumanMessage(content=answer_grader_prompt_formatted)]
)
json.loads(result.content)

{'binary_score': 'yes',
 'explanation': "The student's answer helps to answer the question by providing specific details about the vision models released as part of Llama 3.2. The answer mentions two vision models (Llama 3.2 11B Vision Instruct and Llama 3.2 90B Vision Instruct) that are available on Azure AI Model Catalog, which meets one of the criteria. Additionally, the student provides context about these models being part of Meta's first foray into multimodal AI and their comparison to other visual reasoning models, which is extra information but does not detract from the main point of answering the question. The answer also mentions that these models replace the older text-only Llama 3.1 models, which further supports the student's response. Overall, the student's answer provides sufficient detail to meet all of the criteria and therefore receives a score of 'yes'."}